In [1]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = '/Users/liang/Downloads/spark-1.4.1-bin-hadoop2.6/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.4.1
      /_/

Using Python version 2.7.10 (default, May 28 2015 17:04:42)
SparkContext available as sc, HiveContext available as sqlContext.


In [2]:
%%writefile PageRank.txt
1:2,3
2:3,4
3:5
4:6
5:1,4
6:5

Overwriting PageRank.txt


In [3]:
def computeContribs(urls, rank):
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)
        
def parseDataGraph(line):
    fields = line.split(':')
    return(fields[0], fields[1].split(','))

links = sc.textFile("PageRank.txt").map(parseDataGraph).cache()
ranks = links.map(lambda (url, neighbors): (url, 1.0))
for iteration in xrange(10):
    contribs = links.join(ranks).flatMap(lambda (url, (urls, rank)): computeContribs(urls, rank))
    ranks = contribs.reduceByKey(lambda x,y: x + y).mapValues(lambda rank: rank * 0.85 + 0.15)
print ranks.collect()
sc.stop()

[(u'3', 0.7451283237792715), (u'6', 1.0798511456610982), (u'1', 0.8630224597866966), (u'4', 1.0854830772456592), (u'5', 1.703847287206964), (u'2', 0.5226677063203089)]
